In [44]:
%load_ext autoreload
%autoreload 2

# from __future__ import absolute_import
import sys
sys.path.append("../")

import os
from copy import copy

import numpy as np
import pandas as pd

from random import choice

from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor 
from sklearn.grid_search import RandomizedSearchCV 
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import SelectKBest

from scipy.stats import randint as sp_randint

from utils.evaluation_utils import rmsle
from utils.generic_utils import pickle_out,pickle_in
import utils.preprocessing_utils as prep

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [45]:
data_folder = os.path.join("../","datasets","initial_data_split")

train = pd.read_csv(os.path.join(data_folder,"train.csv"))
valid = pd.read_csv(os.path.join(data_folder,"valid.csv"))

X_train = train.drop(["count","registered","casual"], axis=1)
Y_train = train[["count"]].values.ravel()

X_valid = valid.drop(["count","registered","casual"], axis=1)
Y_valid = valid[["count"]].values.ravel()

print X_train.shape,Y_train.shape
print X_valid.shape,Y_valid.shape
print X_train.head()

(7452, 9) (7452,)
(1722, 9) (1722,)
              datetime  season  holiday  workingday  weather  temp   atemp  \
0  2011-01-01 00:00:00       1        0           0        1  9.84  14.395   
1  2011-01-01 01:00:00       1        0           0        1  9.02  13.635   
2  2011-01-01 02:00:00       1        0           0        1  9.02  13.635   
3  2011-01-01 03:00:00       1        0           0        1  9.84  14.395   
4  2011-01-01 04:00:00       1        0           0        1  9.84  14.395   

   humidity  windspeed  
0        81        0.0  
1        80        0.0  
2        80        0.0  
3        75        0.0  
4        75        0.0  


In [46]:
rolling_lags = [5,10,20,50,300,1000]
edge_gap_varnames = ['edge_lag_%s'%l for l in rolling_lags]
edge_gap_steps = [('add_edge_gap_lag_%s'%lag,prep.AddTimeGaps(lag=lag))for lag in rolling_lags]

max_lag=10
numerical_lagging_variables = ['atemp','humidity','windspeed']
categorical_lagging_variables = ['weather','holiday','workingday','season']
categorical_lagging_varnames = ['%s_lag_%s'%(var,l) for var in categorical_lagging_variables for l in range(1,max_lag,1)]
all_lagging_variables = numerical_lagging_variables + categorical_lagging_variables

lagging_values_steps = [('%s_lag_%s'%(var,lag),
                         prep.LaggingValues(colname = var, lag = max_lag))for var in all_lagging_variables]

stat_calculation_modes = ["whole","day","night"]
median_steps = [('median_%s_%s_%s'%(var,mode,lag),
                 prep.LaggingMedian(colname =var,
                                    lag = lag,
                                    mode=mode)) for lag in rolling_lags
                for var in numerical_lagging_variables 
                for mode in stat_calculation_modes]
max_steps = [('max_%s_%s_%s'%(var,mode,lag),
                 prep.LaggingMax(colname =var,
                                    lag = lag,
                                    mode=mode)) for lag in rolling_lags
                for var in numerical_lagging_variables 
                for mode in stat_calculation_modes]
min_steps =[('min_%s_%s_%s'%(var,mode,lag),
                 prep.LaggingMin(colname =var,
                                    lag = lag,
                                    mode=mode)) for lag in rolling_lags
                for var in numerical_lagging_variables 
                for mode in stat_calculation_modes]
mode_steps =[('mode_%s_%s_%s'%(var,mode,lag),
                 prep.LaggingMode(colname =var,
                                    lag = lag,
                                    mode=mode)) for lag in rolling_lags
                for var in categorical_lagging_variables 
                for mode in stat_calculation_modes]

lagging_mode_varnames = ["%s_%s_mode_%s"%(var,mode,lag) for lag in rolling_lags
                for var in categorical_lagging_variables 
                for mode in stat_calculation_modes]

encode_variables = ['season','weather','holiday','workingday','date_year','day_night']
encode_variables.extend(edge_gap_varnames)
encode_variables.extend(categorical_lagging_varnames)
encode_variables.extend(lagging_mode_varnames)

encoding_steps = [[('encode_label_%s'%var,prep.PandasLabelEncoder(colname=var)),\
        ('encode_one_hot_%s'%var,prep.PandasOneHotEncoder(colname=var))] for var in encode_variables]
encoding_steps = [en_st for sublist in encoding_steps for en_st in sublist ]


prep_steps = [('extract_times', prep.ExtractTimes())]+\
            edge_gap_steps+lagging_values_steps+\
            median_steps+max_steps+min_steps+mode_steps+\
            encoding_steps
#                [('drop_columns', prep.DropColumns(colnames = ["datetime"])),
#               ('feature_selection',SelectKBest(k=100))
#              ]
              
prep_pipe = Pipeline(prep_steps)
X_train = prep_pipe.fit_transform(X_train,Y_train)
X_valid = prep_pipe.transform(X_valid)
print X_train.shape,X_valid.shape

../utils/preprocessing_utils.py:227: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  min_periods=1).median()
../utils/preprocessing_utils.py:279: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  min_periods=1).max()
../utils/preprocessing_utils.py:330: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  min_periods=1).min()
../u

(7452, 635) (1722, 635)


In [47]:
pickle_out(os.path.join("../","datasets","generated_features","train_lagging_all.pkl"),(X_train,Y_train))
pickle_out(os.path.join("../","datasets","generated_features","valid_lagging_all.pkl"),(X_valid,Y_valid))

In [54]:
(X_train,Y_train) = pickle_in(os.path.join("../","datasets","generated_features","train_lagging_all.pkl"))
(X_valid,Y_valid) = pickle_in(os.path.join("../","datasets","generated_features","valid_lagging_all.pkl"))

In [49]:
drop_variables = ['datetime','season','weather','holiday','workingday','date_year','day_night']
X_train=X_train.drop(drop_variables,axis=1)
X_valid=X_valid.drop(drop_variables,axis=1)

In [55]:
print X_train.shape,Y_train.shape,X_valid.shape,Y_valid.shape

(7452, 635) (7452,) (1722, 635) (1722,)


In [26]:
selection_steps = [("rf_selector",prep.RandomForestFeatureSelector(n_estimators = 100,
                                                                   drop_rate = 1,
                                                                   feature_threshold = 10,
                                                                   max_error_increase = 0.01
                                                                  ))
                  ]

select_pipe = Pipeline(selection_steps)
X_train = select_pipe.fit_transform(X_train,Y_train)
X_valid = select_pipe.transform(X_valid)
print X_train.shape,X_valid.shape

Number of features: 14
-9.60706110952
current error: 0.39293889048
Number of features: 13
-0.0173598590601
current error: 0.37557903142
Number of features: 12
0.00777558246123
current error: 0.383354613881
Number of features: 11
-0.00871980776973
current error: 0.374634806111
(7452, 10) (1722, 10)


In [27]:
print X_train.columns.values

['time_hour' 'day_night_1' 'day_night_0' 'day_night' 'atemp_night_max_300'
 'workingday_0' 'date_year' 'humidity' 'temp_day_median_5'
 'atemp_day_median_300']


In [51]:
rf =  RandomForestRegressor(n_jobs=3,oob_score=True,
                            n_estimators=200,
#                             max_features=100,
#                             min_samples_split = 3,
#                             min_samples_leaf=3,
#                             max_depth=25
                           )
rf.fit(X_train,Y_train)

JoblibValueError: JoblibValueError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
/home/jakubczakon/anaconda2/lib/python2.7/runpy.py in _run_module_as_main(mod_name='ipykernel.__main__', alter_argv=1)
    157     pkg_name = mod_name.rpartition('.')[0]
    158     main_globals = sys.modules["__main__"].__dict__
    159     if alter_argv:
    160         sys.argv[0] = fname
    161     return _run_code(code, main_globals, None,
--> 162                      "__main__", fname, loader, pkg_name)
        fname = '/home/jakubczakon/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py'
        loader = <pkgutil.ImpLoader instance>
        pkg_name = 'ipykernel'
    163 
    164 def run_module(mod_name, init_globals=None,
    165                run_name=None, alter_sys=False):
    166     """Execute a module's code without importing it

.....................

In [ ]:
Y_train_pred = rf.predict(X_train)
Y_valid_pred = rf.predict(X_valid)
train_error = rmsle(Y_train_pred,Y_train)
valid_error = rmsle(Y_valid_pred,Y_valid)
print train_error,valid_error
print rf.oob_score_

In [30]:
param_dist = {"max_depth": [15,20,25,30,35,None],
              "max_features": range(1,9,1),
              "min_samples_split": range(1,3,1),
              "min_samples_leaf": range(1,3,1)
              }

rolling_hyperparams = []
with open(os.path.join("../","models","random_forest_simple_hyperparams.txt"),"wb") as f:
    for i in range(50):
        print "Iteration:%s\n"%i
        md = choice(param_dist["max_depth"])
        print "max_depth:%s"%md
        mf = choice(param_dist["max_features"])
        print "max_features:%s"%mf
        ms = choice(param_dist["min_samples_split"])
        print "min_samples_split:%s"%ms
        ml = choice(param_dist["min_samples_leaf"])
        print "min_samples_leaf:%s\n"%ml

        rf = RandomForestRegressor(n_estimators=200,n_jobs=3,verbose=0,
                max_depth = md,
                max_features=mf,
                min_samples_split=ms,
                min_samples_leaf=ml,
               )
        rf.fit(X_train,Y_train)

        Y_train_pred = rf.predict(X_train)
        Y_valid_pred = rf.predict(X_valid)
        train_error = rmsle(Y_train_pred,Y_train)
        valid_error = rmsle(Y_valid_pred,Y_valid)
        print "Train rmsle:%s"%train_error
        print "Train rmsle:%s\n"%valid_error
        f.write("iteration:%s max_depth:%s max_features:%s min_samples_split:%s "\
        "min_samples_leaf:%s "\
        "train_error:%s valid_error:%s\n"%(i,md,mf,ms,ml,train_error,valid_error))
        
        rolling_hyperparams.append([md,mf,ms,ml,train_error,valid_error])
rolling_hyperparams = rolling_hyperparams.vstack(rolling_hyperparams)
pickle_out(os.path.join("../","models","random_forest_simple_hyperparams.pkl"),rolling_hyperparams)

Iteration:0

max_depth:25
max_features:6
min_samples_split:2
min_samples_leaf:2

Train rmsle:0.251414805789
Train rmsle:0.436118448016

Iteration:1

max_depth:15
max_features:8
min_samples_split:1
min_samples_leaf:2

Train rmsle:0.2406344701
Train rmsle:0.423872534794

Iteration:2

max_depth:20
max_features:1
min_samples_split:2
min_samples_leaf:2

Train rmsle:0.4313074541
Train rmsle:0.651272035327

Iteration:3

max_depth:15
max_features:2
min_samples_split:2
min_samples_leaf:2

Train rmsle:0.406570403119
Train rmsle:0.597207053841

Iteration:4

max_depth:None
max_features:3
min_samples_split:1
min_samples_leaf:2

Train rmsle:0.338777142417
Train rmsle:0.535690873446

Iteration:5

max_depth:None
max_features:5
min_samples_split:1
min_samples_leaf:1

Train rmsle:0.178352178661
Train rmsle:0.445361293465

Iteration:6

max_depth:35
max_features:7
min_samples_split:2
min_samples_leaf:2

Train rmsle:0.240863210202
Train rmsle:0.427386749421

Iteration:7

max_depth:25
max_features:7
min_sam

AttributeError: 'list' object has no attribute 'vstack'

In [82]:
number_of_trees = [1,2,3,5,10,50,100,200,300,500,1000]

for nr in number_of_trees:
    rf = RandomForestRegressor(n_estimators=nr,
#             max_features = 9,
#             min_samples_split = 1,
#             min_samples_leaf = 1,
#             max_depth = 20
           )
    rf.fit(X_train, Y_train)
    Y_valid_pred = rf.predict(X_valid)
    valid_score = rmsle(Y_valid_pred,Y_valid)
    print nr, valid_score

1 0.524246358104
2 0.434469563019
3 0.395799150779
5 0.380644031519
10 0.351612307916
50 0.356765725173
100 0.347071524506
200 0.341765233624
300 0.34752623493
500 0.344099122685


KeyboardInterrupt: 

In [218]:
encode_variables = ['season','weather','holiday','workingday','date_year','date_weekday','day_night',
                    'date_month']
encoding_steps = [[('encode_label_%s'%var,prep.PandasLabelEncoder(colname=var)),\
        ('encode_one_hot_%s'%var,prep.PandasOneHotEncoder(colname=var))] for var in encode_variables]
encoding_steps = [en_st for sublist in encoding_steps for en_st in sublist ]

final_steps = [('extract_times', prep.ExtractTimes())]+\
                encoding_steps+\
             [('drop_columns', prep.DropColumns(colnames = ["datetime"])),
             ('random_forest',RandomForestRegressor(n_jobs=3,oob_score=True,
                                                    n_estimators=500,
                                                    max_features=49,
                                                    min_samples_split = 3,
                                                    min_samples_leaf=3,
                                                    max_depth=25))]

final_pipe = Pipeline(final_steps)
final_pipe.fit(X_train,Y_train)

Y_train_pred = final_pipe.predict(X_train)
Y_valid_pred = final_pipe.predict(X_valid)
result_train = rmsle(Y_train_pred,Y_train)
result_valid = rmsle(Y_valid_pred,Y_valid)
print result_train
print result_valid

0.231232339306
0.281295493245


In [219]:
model_filepath = os.path.join("../","models","random_forest_weekdays_pipeline.pkl")
pickle_out(model_filepath,final_pipe,compresion_mode=5)